In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc.types import FrequencySeries
from pycbc.noise import noise_from_psd
from pycbc.psd import aLIGOZeroDetHighPower
from scipy.stats import norm, skew, kurtosis
from joblib import Parallel, delayed

# 参数设置
total_segments = 50000
noise_segments = 40000  # 仅包含噪声的段数
signal_segments = total_segments - noise_segments  # 包含chirp信号的段数
sampling_rate = 4096  # 采样率（每秒采样数）
duration = 1  # 每段数据的持续时间（秒）
delta_t = 1 / sampling_rate  # 每个采样点的时间间隔
low_freq_cutoff = 20  # 低频截止频率（Hz）
target_frequency = 100  # 分析的固定频率（Hz）

# Step 1: 生成高斯白噪声数据
psd = aLIGOZeroDetHighPower(sampling_rate // 2 + 1, delta_f=1.0 / duration, low_freq_cutoff=low_freq_cutoff)
noise_data = noise_from_psd(total_segments * int(duration * sampling_rate), delta_t, psd)

# Step 2: 提取固定频率分量
freq_data = noise_data.to_frequencyseries()
target_values = np.abs(freq_data[target_frequency::sampling_rate])[:noise_segments]

# Step 3: 生成简化的 chirp 信号，并提取固定频率分量
chirp_signal_freq = 1.0  # 简化为一个固定频率的 chirp 信号
signal_data = np.random.normal(0, 1, signal_segments) + chirp_signal_freq

# Step 4: 混合包含信号和仅噪声的数据段
combined_data = np.concatenate([target_values, signal_data])
np.random.shuffle(combined_data)  # 随机混合数据

# Step 5: 滑动窗口检测交叉相关（并行优化版）
# 使用滑动窗口方法对交叉相关进行局部评估
window_size = 1024  # 滑动窗口大小
step_size = 512  # 窗口步长

# 定义滑动窗口计算函数
def calculate_window_snr(start_index):
    window = combined_data[start_index:start_index + window_size]
    window_mean = np.mean(window)
    window_variance = np.var(window)
    return window_mean / (np.sqrt(window_variance) + 1e-6)

# 使用并行计算滑动窗口的SNR值
cross_corr_snr_values = Parallel(n_jobs=-1)(
    delayed(calculate_window_snr)(start) for start in range(0, len(combined_data) - window_size + 1, step_size)
)

max_snr = max(cross_corr_snr_values)
print(f"Maximum SNR from Sliding Window: {max_snr}")

# Step 6: 阈值检测并提取信号（改进版）
# 设置信噪比阈值（降低阈值以提高灵敏度）
snr_threshold = 1.5  # 降低阈值以捕获更多可能的信号

# 检测信号
if max_snr > snr_threshold:
    print("Signal detected with significant SNR using sliding window.")
    detected_signal = max_snr  # 可以进一步提取信号
else:
    print("No significant signal detected using sliding window.")
    detected_signal = None

# Step 7: 可视化滑动窗口的 SNR 结果
plt.figure(figsize=(12, 6))
plt.plot(cross_corr_snr_values, label="Sliding Window SNR Values")
plt.xlabel("Window Index")
plt.ylabel("SNR")
plt.title("SNR Values from Sliding Window Analysis")
plt.legend()
plt.show()

# Step 8: 计算偏度和峰度结果
signal_data_skewness = skew(signal_data)
signal_data_kurtosis = kurtosis(signal_data)
print(f"Skewness of Signal Data: {signal_data_skewness}")
print(f"Kurtosis of Signal Data: {signal_data_kurtosis}")

# Step 9: 构造最大似然函数并计算（基于改进的处理方式）
def likelihood_max_likelihood(snr_values, skewness, kurtosis):
    # 使用滑动窗口的SNR值构造似然函数，结合偏度和峰度信息
    gaussian_likelihood = norm.pdf(np.mean(snr_values), loc=0, scale=np.std(snr_values))
    skew_term = np.exp(-0.5 * skewness ** 2 / (np.var(snr_values) + 1e-6))
    kurtosis_term = np.exp(-0.5 * kurtosis ** 2 / (np.var(snr_values) + 1e-6))
    return gaussian_likelihood * skew_term * kurtosis_term

# 计算最大似然值
likelihood_value = likelihood_max_likelihood(cross_corr_snr_values, signal_data_skewness, signal_data_kurtosis)
print(f"Maximum Likelihood Value: {likelihood_value}")

# Step 10: 贝叶斯分析 - 使用充分统计量计算后验概率分布（改进版）
x = np.linspace(-5, 5, 1000)  # 根据 SNR 的范围调整
posterior_prob = norm.pdf(x, loc=max_snr, scale=np.sqrt(np.var(cross_corr_snr_values) + 1e-6))

# 使用一个合适的权重调整后验概率
weight = max_snr / (snr_threshold + 1e-6)
if weight > 1:
    weight = 1

posterior_prob_adjusted = posterior_prob * weight

# 归一化后验概率
posterior_prob_adjusted /= np.sum(posterior_prob_adjusted) * (x[1] - x[0])

# Step 11: 可视化后验概率分布
plt.figure(figsize=(12, 6))
plt.plot(x, posterior_prob_adjusted, label="Adjusted Posterior Probability Distribution using Sufficient Statistics")
plt.xlabel("Signal Strength")
plt.ylabel("Probability Density")
plt.title("Adjusted Posterior Probability Distribution of Signal Strength")
plt.legend()
plt.show()
